In [1]:
import numpy as np
import pulp

# Problème du Voyageur de Commerce (TSP)
---
## Variables
$$
x_{ij} =
\begin{cases}
1 & \text{si le trajet va de la ville } i \text{ à la ville } j, \\
0 & \text{sinon.}
\end{cases}
$$

## Fonction objectif
$$
\min \sum_{i=1}^{n} \sum_{\substack{j=1 \\ j \neq i}}^{n} c_{ij} \, x_{ij}
$$
où ($c_{ij}$) est le coût ou la distance entre la ville (i) et la ville (j).

## Contraintes

### 1/ Contraintes de départ et d’arrivée
$$
\sum_{\substack{j=1 \\ j \neq i}}^{n} x_{ij} = 1, \quad \forall i = 1,\dots,n
$$

$$
\sum_{\substack{i=1 \\ i \neq j}}^{n} x_{ij} = 1, \quad \forall j = 1,\dots,n
$$

### 2/ Contraintes d’intégralité
$$
x_{ij} \in \{0,1\}, \quad \forall i,j = 1,\dots,n
$$

### 3/ Contraintes de sous-tours (Miller–Tucker–Zemlin)

Soit $u_i$ une variable auxiliaire associée à la ville $i$ ($i = 2,\dots,n$), et $n$ le nombre total de villes.  

Pour éliminer les sous-tours :

$$
u_i - u_j + n \, x_{ij} \le n - 1, \quad \forall i \neq j, \; i,j = 2,\dots,n
$$

avec :  
- $x_{ij} \in \{0,1\}$ : 1 si le trajet va de $i$ à $j$, 0 sinon  
- $u_i \ge 0$ et $u_i \le n-1$

---

## Création des villes et de la matrice adjacente avec les couts

In [2]:
n = 30

villes = [i for i in range(n)]
np.random.seed(32)

C = np.random.randint(1, 100, size=(n, n))
C = (C + C.T) // 2
np.fill_diagonal(C, 0)

print(C)


[[ 0 28 49 63 56 48 47 47 89 28 10 24  9 59 33 81 28 62 56 46 88 42 50 27
  56 62 56 55 36 37]
 [28  0 39 76 71 46 38 54 36 87 65 70 56 46 61 35 80 30 59 26 39 32 37 57
  47 35 75 63 96 56]
 [49 39  0 62 63 75 59 59 55 82 14 54 11 47 47 57 34 12 70 38 44 25 23 44
  73 71 67 46 85 44]
 [63 76 62  0 78 94 30 88 44 61 49 31 42 32 63 76 69 58 70 40 81 49 32 36
  52 87 50 61 72 68]
 [56 71 63 78  0 43 72 60 64 30 55 65 13 66 18 56 80 20 72 62 27 42 13 64
  10 45 50 44 79 69]
 [48 46 75 94 43  0 56 43 50 45 25 39 71 48 20 14 11 69 58 42 28 51 58 46
  24 44 64 46 45 61]
 [47 38 59 30 72 56  0 35 35 52 62 52 70 52 46 51 63 29 45 21 25 38 20 37
  64 51 65 29 67 30]
 [47 54 59 88 60 43 35  0 25 35 66 46 50 29 66 29 52 59 34 54 55 25 25 60
  44 45 38 79 37 39]
 [89 36 55 44 64 50 35 25  0 89 51 23 41 14 36 77 54 71 20 34 50 38 29 12
  39 64 35 84 78 76]
 [28 87 82 61 30 45 52 35 89  0 44 20 22 49 87 52 91 39 60 50 22 38 76 53
  36 54 69 41 34 18]
 [10 65 14 49 55 25 62 66 51 44  0 15  4 68 30 53 

## Définition des variables et de la fonction objectif

In [3]:
n = len(villes)

x = {
    (i, j): pulp.LpVariable(f"x_{i}_{j}", cat="Binary") 
    for i in villes 
    for j in villes 
    if i != j
}

prob = pulp.LpProblem("TSP", pulp.LpMinimize)

prob += pulp.lpSum(
    C[i, j] * x[i, j]
    for i in villes
    for j in villes
    if i != j
)

## Création des contraintes

In [4]:
n = len(villes)

# Contraintes de départ et d’arrivée
for i in range(n):
    prob += pulp.lpSum(
        x[i, j]
        for j in villes
        if j != i
    ) == 1

    prob += pulp.lpSum(
        x[j, i]
        for j in villes
        if j != i
    ) == 1

# Contraintes de sous-tours (Miller–Tucker–Zemlin)
u = pulp.LpVariable.dicts("u", villes, lowBound=0, upBound=n - 1, cat="Integer")

prob += u[0] == 0

for i in villes[1:]:
    for j in villes[1:]:
        if i != j:
            prob += u[i] - u[j] + n * x[i, j] <= n - 1


## Résolution du problème

In [5]:
prob.solve()

print("Valeur optimale:", pulp.value(prob.objective))

chemin = [0]

while len(chemin) < n:
    i = chemin[-1]
    for j in villes:
        if i != j and pulp.value(x[i, j]) > 0.5:
            chemin.append(j)
            break
            
chemin.append(0)

print(pulp.LpStatus[prob.status])
print(chemin)

Valeur optimale: 538.0
Optimal
[0, 10, 28, 15, 18, 26, 16, 5, 14, 4, 24, 23, 8, 13, 27, 22, 25, 1, 19, 6, 3, 11, 20, 9, 29, 7, 21, 17, 2, 12, 0]
